In [0]:
%sql

-- Databricks: 03_kpis_gold.sql
-- Purpose: Build Gold KPI views from Silver layer.
-- Inputs (must exist):
--   silver_payments(payment_id, customer_id, amount, status, method, ts, month)
--   silver_chargebacks(payment_id, reason, outcome, opened_ts, resolved_ts, month_opened)

----------------------------------------------------------------------
-- Choose your catalog/schema
-- Non-UC (default):
USE demo_finance;

-- Unity Catalog (uncomment if UC is enabled):
-- USE CATALOG main;
-- USE SCHEMA demo_finance;
----------------------------------------------------------------------

-- 1) Monthly payments summary (counts + gross revenue)
CREATE OR REPLACE VIEW gold_payments_monthly AS
SELECT
  month,
  COUNT(*) AS payments,
  SUM(CASE WHEN status = 'settled' THEN 1 ELSE 0 END) AS settled,
  SUM(CASE WHEN status = 'failed'  THEN 1 ELSE 0 END) AS failed,
  SUM(CASE WHEN status = 'settled' THEN amount ELSE 0 END) AS gross_revenue
FROM silver_payments
GROUP BY month;

COMMENT ON VIEW gold_payments_monthly IS
'Gold KPI: monthly payment counts and gross revenue aggregated from silver_payments.';

-- 2) Monthly chargeback rate (chargebacks ÷ settled)
CREATE OR REPLACE VIEW gold_chargeback_rate AS
WITH cbs AS (
  SELECT month_opened AS month, COUNT(*) AS chargebacks
  FROM silver_chargebacks
  GROUP BY month_opened
),
pays AS (
  SELECT month, SUM(CASE WHEN status = 'settled' THEN 1 ELSE 0 END) AS settled
  FROM silver_payments
  GROUP BY month
)
SELECT
  COALESCE(p.month, c.month) AS month,
  COALESCE(c.chargebacks, 0) AS chargebacks,
  COALESCE(p.settled, 0)     AS settled,
  CASE WHEN COALESCE(p.settled, 0) = 0 THEN 0
       ELSE ROUND(COALESCE(c.chargebacks, 0) / p.settled, 4)
  END AS chargeback_rate
FROM pays p
FULL OUTER JOIN cbs c
  ON p.month = c.month;

COMMENT ON VIEW gold_chargeback_rate IS
'Gold KPI: monthly chargebacks and chargeback rate (chargebacks / settled).';

-- 3) Convenience: single KPI view joining both
CREATE OR REPLACE VIEW gold_kpi_monthly AS
SELECT
  COALESCE(p.month, r.month)                         AS month,
  p.payments,
  p.settled,
  p.failed,
  p.gross_revenue,
  r.chargebacks,
  r.chargeback_rate,
  CASE WHEN COALESCE(p.payments, 0) = 0 THEN 0
       ELSE ROUND(COALESCE(p.failed, 0) / p.payments, 4)
  END AS failure_rate
FROM gold_payments_monthly p
FULL OUTER JOIN gold_chargeback_rate r
  ON p.month = r.month;

COMMENT ON VIEW gold_kpi_monthly IS
'Gold KPI: unified monthly KPIs (payments, settled, failed, gross_revenue, chargebacks, chargeback_rate, failure_rate).';

-- 4) Sanity checks (safe to keep here; useful when running interactively)
-- (You can comment these out for automated jobs.)
SELECT * FROM gold_payments_monthly  ORDER BY month LIMIT 12;
SELECT * FROM gold_chargeback_rate   ORDER BY month LIMIT 12;
SELECT * FROM gold_kpi_monthly       ORDER BY month LIMIT 12;
